<a href="https://colab.research.google.com/github/itmaster1/Info_Site/blob/master/Hot_news_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os
import pandas

date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'


dt_index = pandas.date_range(start='20211026', end='20211027')
# pandas.date_range(start='20160901', end='20161031',freq='W-MON')
# 을 하면 해당 기간 매주 월요일들만 추출합니다.
dt_list = dt_index.strftime("%Y%m%d").tolist()

news_dict = {}
idx = 0
cur_page = 0

for d in dt_list:
    url = 'https://finance.naver.com/news/news_list.naver?mode=RANK&date={}'.format(d)
    print(url)
    req = requests.get(url) # 가장많인 본 뉴스의 최근일자 일경우 ( 레벨이 5단계가 아닌경우 에러발생 )
    soup = BeautifulSoup(req.text, 'html.parser')

    page = 0 #일자별 page 초기화 
    # 마지막 페이지 숫자 가져오기 
    el_table_navi = soup.find("table", class_="Nnavi")
    el_td_last = el_table_navi.find("td", class_="pgRR")
    pg_last = el_td_last.a.get('href').rsplit('page')[1]
    pg_last = pg_last.split('=')[1]
    page = int(pg_last)

    print()
    print('크롤링 중...')

    for i in range(1,page+1):
        print(i)    
        url_1 = url+"&page="+format(i)
        #print(url_1)
        #req_1 = requests.get('https://finance.naver.com/news/news_list.naver?mode=RANK&date=&date='+str(d) +'&page=&p=' + str(i))
        req_1 = requests.get(url_1)
        #req_1 = requests.get('https://finance.naver.com/news/news_list.naver?mode=RANK&date=20211026&page=2')
        soup = BeautifulSoup(req_1.text, 'html.parser')

        #for href in soup.find("ul", class_="newsList").find_all("li"): #newsList
        for href in soup.find("div", class_="hotNewsList").find_all("li"):
            #news_dict[idx] = href.find("a")["title"]
            #print(href)
            news_dict[idx] = {'seq' : idx+1
                            ,'date': re.sub(r'[^0-9]' , '',str(href.find("span",{'class' : 'wdate'}))), 
                            'title' : href.find("a")["title"],
                            'url' : href.find("a")["href"], # https://finance.naver.com + 해야함                         
                            'source': re.sub('[a-zA-Z\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"\♥\♡\ㅋ\ㅠ\ㅜ\ㄱ\ㅎ\ㄲ\ㅡ]' , '',str(href.find("span",{'class' : 'press'})))}
            idx += 1

print('크롤링 완료')

print('데이터프레임 변환')
news_df = DataFrame(news_dict).T

news_df

#mysql 처리 
# https://hyun-am-coding.tistory.com/entry/%ED%81%AC%EB%A1%A4%EB%A7%81%ED%95%9C-%EB%8D%B0%EC%9D%B4%ED%84%B0-DB%EC%97%90-%EC%A0%80%EC%9E%A5%ED%95%98%EA%B8%B0


In [ ]:
import sqlite3
print(sqlite3.version)
print(sqlite3.sqlite_version)

In [ ]:
# DB 생성 (오토 커밋)
conn = sqlite3.connect("hot_news_data.db", timeout=10)
# 커서 획득
c = conn.cursor()

# 실행할 때마다 다른값이 나오지 않게 테이블을 제거해두기 
c.execute("DROP TABLE IF EXISTS news")

# 테이블 생성 (데이터 타입은 TEST, NUMERIC, INTEGER, REAL, BLOB 등)
c.execute("CREATE TABLE IF NOT EXISTS news \
    (seq int PRIMARY KEY, news_date text  , title text, new_url text, news_source text)")

# 데이터 저장하기
for row in news_df.itertuples():
    sql = "insert into news(seq, news_date, title, new_url,news_source) values(?,?,?,?,?)"
    c.execute(sql,(row[1],row[2],row[3],row[4],row[5]))

# 커밋하기 
conn.commit() 
# 연결종료하기 
conn.close()

In [ ]:
# 데이터 조회(전체) 
# DB 생성 (오토 커밋)
conn = sqlite3.connect("hot_news_data.db", timeout=10)
# 커서 획득
c = conn.cursor()

c.execute("SELECT * FROM news")

rows = c.fetchall() 
for row in rows: 
    print('retrieve1 >', row) # 조회 없음

# 커밋하기 
conn.commit() 
# 연결종료하기 
conn.close()